In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,81437,3,4.0,1,2.30,0.0155,...,4,0,1,1,-1.00,1,0.000000,0.0,0.000000,0
1,1,12,187,219,111106,3,2.5,1,0.69,0.0071,...,4,0,1,1,-1.00,1,0.000000,0.0,0.000000,0
2,1,12,187,219,114766,2,3.5,1,1.61,0.0052,...,4,0,1,1,-1.00,1,0.000000,0.0,-1.000000,0
3,1,12,187,219,74474,3,4.5,1,2.40,0.1255,...,4,0,1,1,-1.00,1,0.000000,0.0,0.000000,0
4,1,12,187,219,95307,4,3.5,1,2.40,0.1149,...,4,0,1,1,-1.00,1,-0.500000,28.0,0.333333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332784,24,219,100,115562,4,4.0,1,4.74,0.0691,...,1,0,1,0,328.83,0,0.000000,0.0,0.000000,0
495830,332784,24,219,100,28943,4,3.5,1,4.84,0.0722,...,1,0,1,0,329.96,0,0.333333,7.0,0.000000,0
495831,332784,24,219,100,140304,3,4.0,0,4.55,0.1606,...,1,0,1,0,328.93,0,0.000000,0.0,0.000000,0
495832,332784,24,219,100,25223,4,4.0,0,4.96,0.0857,...,1,0,1,0,329.49,0,0.000000,0.0,0.000000,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
# drop search id's
x.drop('srch_id', axis = 1,inplace=True)
x_val.drop('srch_id', axis = 1, inplace=True)

In [6]:
# TODO get categoric columns
#np.set_printoptions(threshold=np.inf)

In [7]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=1000) 

#label_gain

##rank_xendcg, XE_NDCG_MART ranking objective function, aliases: xendcg, xe_ndcg, xe_ndcg_mart, xendcg_mart
#rank_xendcg is faster than and achieves the similar performance as lambdarank

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000
[1]	valid_0's ndcg@1: 0.93978	valid_0's ndcg@2: 0.95798	valid_0's ndcg@3: 0.966405	valid_0's ndcg@4: 0.970089	valid_0's ndcg@5: 0.971615
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.941773	valid_0's ndcg@2: 0.959871	valid_0's ndcg@3: 0.968021	valid_0's ndcg@4: 0.971556	valid_0's ndcg@5: 0.972884
[3]	valid_0's ndcg@1: 0.943375	valid_0's ndcg@2: 0.960964	valid_0's ndcg@3: 0.968925	valid_0's ndcg@4: 0.972303	valid_0's ndcg@5: 0.973602
[4]	valid_0's ndcg@1: 0.943474	valid_0's ndcg@2: 0.961212	valid_0's ndcg@3: 0.969044	valid_0's ndcg@4: 0.972425	valid_0's ndcg@5: 0.973703
[5]	valid_0's ndcg@1: 0.94404	valid_0's ndcg@2: 0.961484	valid_0's ndcg@3: 0.96926	valid_0's ndcg@4: 0.972631	valid_0's ndcg@5: 0.973885
[6]	valid_0's ndcg@1: 0.944046	valid_0's ndcg@2: 0.961523	valid_0's ndcg@3: 0.969264	valid_0's ndcg@4: 0.972661	valid_0's ndc

[59]	valid_0's ndcg@1: 0.945206	valid_0's ndcg@2: 0.962543	valid_0's ndcg@3: 0.97011	valid_0's ndcg@4: 0.97333	valid_0's ndcg@5: 0.974539
[60]	valid_0's ndcg@1: 0.945177	valid_0's ndcg@2: 0.962537	valid_0's ndcg@3: 0.970077	valid_0's ndcg@4: 0.973315	valid_0's ndcg@5: 0.974521
[61]	valid_0's ndcg@1: 0.945229	valid_0's ndcg@2: 0.962529	valid_0's ndcg@3: 0.970093	valid_0's ndcg@4: 0.97333	valid_0's ndcg@5: 0.974541
[62]	valid_0's ndcg@1: 0.945258	valid_0's ndcg@2: 0.962577	valid_0's ndcg@3: 0.970111	valid_0's ndcg@4: 0.973328	valid_0's ndcg@5: 0.974556
[63]	valid_0's ndcg@1: 0.945298	valid_0's ndcg@2: 0.962544	valid_0's ndcg@3: 0.970122	valid_0's ndcg@4: 0.973326	valid_0's ndcg@5: 0.974558
[64]	valid_0's ndcg@1: 0.945286	valid_0's ndcg@2: 0.962554	valid_0's ndcg@3: 0.970111	valid_0's ndcg@4: 0.973318	valid_0's ndcg@5: 0.974557
[65]	valid_0's ndcg@1: 0.945338	valid_0's ndcg@2: 0.962611	valid_0's ndcg@3: 0.970135	valid_0's ndcg@4: 0.973359	valid_0's ndcg@5: 0.974581
[66]	valid_0's ndcg@1: 

[118]	valid_0's ndcg@1: 0.945303	valid_0's ndcg@2: 0.96275	valid_0's ndcg@3: 0.970191	valid_0's ndcg@4: 0.973405	valid_0's ndcg@5: 0.974609
[119]	valid_0's ndcg@1: 0.945321	valid_0's ndcg@2: 0.962754	valid_0's ndcg@3: 0.970196	valid_0's ndcg@4: 0.973411	valid_0's ndcg@5: 0.974616
[120]	valid_0's ndcg@1: 0.945321	valid_0's ndcg@2: 0.962773	valid_0's ndcg@3: 0.970208	valid_0's ndcg@4: 0.97341	valid_0's ndcg@5: 0.974619
[121]	valid_0's ndcg@1: 0.945292	valid_0's ndcg@2: 0.962782	valid_0's ndcg@3: 0.970203	valid_0's ndcg@4: 0.973404	valid_0's ndcg@5: 0.974614
[122]	valid_0's ndcg@1: 0.945315	valid_0's ndcg@2: 0.962763	valid_0's ndcg@3: 0.970203	valid_0's ndcg@4: 0.973405	valid_0's ndcg@5: 0.97461
[123]	valid_0's ndcg@1: 0.945332	valid_0's ndcg@2: 0.962749	valid_0's ndcg@3: 0.970223	valid_0's ndcg@4: 0.973411	valid_0's ndcg@5: 0.974616
[124]	valid_0's ndcg@1: 0.945343	valid_0's ndcg@2: 0.962778	valid_0's ndcg@3: 0.970214	valid_0's ndcg@4: 0.973406	valid_0's ndcg@5: 0.97462
[125]	valid_0's n

[178]	valid_0's ndcg@1: 0.945469	valid_0's ndcg@2: 0.96276	valid_0's ndcg@3: 0.970273	valid_0's ndcg@4: 0.973469	valid_0's ndcg@5: 0.97466
[179]	valid_0's ndcg@1: 0.945498	valid_0's ndcg@2: 0.962751	valid_0's ndcg@3: 0.97028	valid_0's ndcg@4: 0.973475	valid_0's ndcg@5: 0.974666
[180]	valid_0's ndcg@1: 0.945463	valid_0's ndcg@2: 0.962749	valid_0's ndcg@3: 0.970273	valid_0's ndcg@4: 0.973469	valid_0's ndcg@5: 0.974659
[181]	valid_0's ndcg@1: 0.945469	valid_0's ndcg@2: 0.962746	valid_0's ndcg@3: 0.970286	valid_0's ndcg@4: 0.973466	valid_0's ndcg@5: 0.974662
[182]	valid_0's ndcg@1: 0.945457	valid_0's ndcg@2: 0.962756	valid_0's ndcg@3: 0.970269	valid_0's ndcg@4: 0.97346	valid_0's ndcg@5: 0.974656
[183]	valid_0's ndcg@1: 0.945452	valid_0's ndcg@2: 0.962755	valid_0's ndcg@3: 0.970276	valid_0's ndcg@4: 0.973453	valid_0's ndcg@5: 0.974656
[184]	valid_0's ndcg@1: 0.945429	valid_0's ndcg@2: 0.962747	valid_0's ndcg@3: 0.970268	valid_0's ndcg@4: 0.973453	valid_0's ndcg@5: 0.974652
[185]	valid_0's n

LGBMRanker(metric='ndcg', num_boost_round=1000, objective='lambdarank')

In [8]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [9]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [40]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test_srch_ids = test.srch_id
test.drop(['date_time', 'srch_id'], axis=1, inplace=True)

In [41]:
predictions = model.predict(test)

In [42]:
len(predictions)

4959183

In [43]:
submission = test_srch_ids.copy()
del test_srch_ids
gc.collect()
submission = pd.concat([submission, test['prop_id']], axis=1)
submission['pred'] = predictions

In [45]:
del test
gc.collect()

40

In [46]:
submission.head(n=100)

,srch_id,prop_id,pred
0,1,3180,0.034319
1,1,5543,0.049951
2,1,14142,0.092389
3,1,22393,-0.299970
4,1,24194,0.984675
...,...,...,...
95,10,11203,-0.636216
96,10,23777,-0.988582
97,10,31281,0.918808
98,10,37420,-1.358286


In [47]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [48]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,1.604440
9,1,54937,1.332769
12,1,61934,1.137356
4,1,24194,0.984675
5,1,28181,0.918229
6,1,34263,0.766034
8,1,50162,0.694907
18,1,82231,0.554539
20,1,90385,0.508035
13,1,63894,0.494229


In [49]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)